In [ ]:
!pip install transformers
!pip install git+https://github.com/huggingface/peft.git
!pip install bitsandbytes
!pip install evaluate

## Imports  

In [5]:
#General imports
import numpy as np 
import pandas as pd 
import gdown 
import os
import requests
import string
import re 
import shutil
from utils import *


#Related to transformers and models

from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch 
import torch.nn as nn


#Result imports
import matplotlib.pyplot as plt

ImportError: cannot import name 'DataCollatorForSequenceClassification' from 'transformers' (/usr/local/lib/python3.8/dist-packages/transformers/__init__.py)

## Descarga del modelo a fine tunear 

In [4]:
model_name = "bhadresh-savani/bert-base-go-emotion"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForTopClassification.from_pretrained("bhadresh-savani/bert-base-go-emotion")

NameError: name 'AutoModelForSequenceClassification' is not defined

## Descarga y tratamiento de los datos 

In [ ]:
#Fichero se encuentra en enlace externo
url = "https://raw.githubusercontent.com/PoorvaRane/Emotion-Detector/master/ISEAR.csv"
output_file = "ISEAR.csv"

destination_folder = "data"
#Si no existe la carpeta en la que queremos guardar los datos debemos crearla
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

response = requests.get(url)
with open(output_file, 'wb') as f:
    f.write(response.content)

shutil.move(output_file, f"{destination_folder}/{output_file}")

# Load and preprocess the dataset
df = load_and_preprocess_data('./data/ISEAR.csv')
df['Emotion'] = df['Emotion'].replace('guit', 'guilt')


## Probamos el modelo a ver de cuantas clases dispone  

In [3]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
resultados = pipe("fuck you leave me alone")
print(resultados)
accum = 0
for elemento in resultados: 
    for cuenta in elemento:
        accum = accum + 1 
print(accum)

NameError: name 'TextClassificationPipeline' is not defined

Como observamos en este modelo se tienen 28 clases (clásico del dataset de goemotion), ahora tenemos varias opciones, podemos intentar modificar esta ultima capa para poder trabajar con los logits, lo cual numéricamente es más estable o añadir una capa final de clasificación después de esta para clasificar entre las emociones de ekman que estamos utilizando en nuestro estudio. 

No sabemos el método que han usado para clasificar las emociones. Por tanto lo primero que vamos a hacer va a ser realizar un entrenamiento básico añadiendo una capa final para clasificar lo que nos interesa

## Exploración del modelo 

In [ ]:
print(model.config)
print(model.num_labels)

El objeto `BertConfig` contiene los parámetros y configuraciones necesarios para el modelo BERT utilizado en la clasificación de secuencias con múltiples etiquetas. A continuación se presenta una explicación de los campos clave en la configuración:

- `hidden_size` y `num_attention_heads`: Estos parámetros determinan la dimensionalidad de los vectores ocultos y el número de cabezas de atención en el modelo BERT, respectivamente.

- `num_hidden_layers`: Indica el número de capas ocultas en el modelo. Aumentar este valor puede aumentar la capacidad del modelo para aprender representaciones más complejas.

- `dropout`: Controla la tasa de abandono (dropout) aplicada a las salidas de las capas ocultas para evitar el sobreajuste y mejorar la generalización.

- `initializer_range`: Especifica el rango de inicialización para los pesos del modelo.

- `max_position_embeddings`: Define la longitud máxima permitida de las secuencias de entrada.

- `vocab_size`: Indica el tamaño del vocabulario utilizado por el modelo.

Estos campos son cruciales para configurar correctamente el modelo BERT en la clasificación de secuencias con múltiples etiquetas. Sin embargo, también debemos considerar otros aspectos como el modelo preentrenado, el conjunto de datos y los hiperparámetros específicos del entrenamiento.


In [ ]:
texto = "I love this movie!"
inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)

# Obtener los logits
logits = outputs.logits

print(logits)

Ahora queremos obtener cual es la clase que es más probable que se haya predicho

In [ ]:
probabilidades = torch.softmax(logits, dim=1)

# Obtener la clase con la probabilidad más alta
clase_predicha_idx = np.argmax(probabilidades.detach().numpy())
clase_predicha = model.config.id2label[clase_predicha_idx]

print("Clase predicha (índice):", clase_predicha_idx)
print("Clase predicha (texto):", clase_predicha)

## Creación de la clase  

In [ ]:
import torch.nn as nn

class MiModelo(nn.Module):
    def __init__(self, modelo_base, num_clases):
        super(MiModelo, self).__init__()
        
        # Definimos el modelo base que estará preentrenado
        self.modelo_base = modelo_base

        # Capa de dropout para evitar sobreajuste, con probabilidad de dropout de 0.5
        self.dropout = nn.Dropout(0.5)
        
        # Capa totalmente conectada que transforma la salida del modelo base en una salida de tamaño 14
        self.fc1 = nn.Linear(self.modelo_base.config.hidden_size, 14)
        
        # Función de activación ReLU
        self.activacion = nn.ReLU()
        
        # Segunda capa totalmente conectada que transforma la salida de tamaño 14 a la cantidad de clases
        self.fc2 = nn.Linear(14, num_clases)

    def forward(self, input_ids, attention_mask):
        # Pasamos los inputs a través del modelo base
        outputs = self.modelo_base(input_ids=input_ids, attention_mask=attention_mask)

        # Tomamos la salida del modelo base (output del token de clasificación)
        pooled_output = outputs.pooler_output
        
        # Aplicamos dropout a la salida del modelo base
        pooled_output = self.dropout(pooled_output)
        
        # Pasamos la salida por la primera capa completamente conectada
        hidden = self.fc1(pooled_output)
        
        # Aplicamos la función de activación
        hidden = self.activacion(hidden)
        
        # Pasamos la salida por la segunda capa completamente conectada para obtener los logits finales
        logits = self.fc2(hidden)
        
        return logits

    def training_step(self, inputs, targets):
        # Obtenemos los logits pasando los inputs por el modelo
        logits = self.forward(inputs.input_ids, inputs.attention_mask)
        
        # Definimos la función de pérdida (Cross Entropy Loss)
        loss_fn = nn.CrossEntropyLoss()
        
        # Calculamos la pérdida
        loss = loss_fn(logits, targets)
        
        return loss


In [ ]:
num_clases = 7
mi_modelo = MiModelo(modelo_base=model, num_clases=num_clases)


## Analisis de mejoras usando Peft 

In [ ]:
lr = 1e-3
batch_size = 16
num_epochs = 10


tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all"
)



In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()